# 개요
- pykrx를 통해 수집한 데이터에 대한 전체 FG score 계산수행

In [1]:
%run ../Utils/FGindex_function.ipynb
%run ../Utils/portfolio_metrics.ipynb
%run ../Utils/Backtesting_function.ipynb

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from operator import itemgetter
from datetime import date, datetime

In [3]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [5]:
ticker = '1244'

In [29]:

# stock preprocessing
stock_df = pd.read_csv("../DATA/data_index/{}.csv".format(ticker))
stock_df = stock_df[stock_df[stock_df['거래량']!=0].index[0]:].reset_index(drop=True)

# FG_score
# FG_score = FG_index(stock_df)
# FG_score.to_csv("../data_FGscore/{}.csv".format(ticker))


In [30]:
stock = stock_df

In [31]:
stock

,날짜,시가,고가,저가,종가,거래량,거래대금,상장시가총액
0,2020-04-27,2290.60,2307.84,2288.62,2306.94,533609260,2599623519268,133611951774326
1,2020-04-28,2321.31,2328.59,2295.21,2312.13,907960556,4126013583631,133915621414620
2,2020-04-29,2318.07,2330.58,2316.96,2327.17,1757597499,3663602922859,134787360028365
3,2020-05-04,2300.04,2339.88,2296.52,2321.39,935001033,3114399361165,134444396935841
4,2020-05-06,2343.88,2356.23,2336.96,2356.23,928402712,2932569784030,136462967951571
...,...,...,...,...,...,...,...,...
628,2022-11-11,2942.22,2964.23,2937.93,2951.97,307688925,2676187101257,170070798215184
629,2022-11-14,2958.85,2985.08,2956.98,2973.63,340500264,1782589489557,171326447025923
630,2022-11-15,2980.15,3001.23,2978.84,3001.22,423046143,2653033720344,172905933873296
631,2022-11-16,3010.35,3014.98,2982.92,3013.87,443440944,2579048829171,173620505357300


In [16]:
final_data

,date,close,FG_score
365,2021-10-20,4165.55,0.514277
366,2021-10-21,4137.81,0.499877
367,2021-10-22,4129.89,0.495466
368,2021-10-25,4152.00,0.505636
369,2021-10-26,4174.90,0.513080
...,...,...,...
628,2022-11-11,2951.97,0.548805
629,2022-11-14,2973.63,0.560976
630,2022-11-15,3001.22,0.576010
631,2022-11-16,3013.87,0.576050


In [34]:
final_data

,date,close,volume,multiple_volume,per,Volatility_list,Volume_list,S1_score,S2_score,FG_score
365,2021-10-20,4165.55,547853785,2.282112e+12,-0.192879,-0.192879,-0.192879,0.521780,0.109479,0.514277
366,2021-10-21,4137.81,747287355,3.092133e+12,-0.665938,-0.665938,-0.665938,0.548735,-0.000895,0.499877
367,2021-10-22,4129.89,584002058,2.411864e+12,-0.191406,-0.191406,-0.191406,0.511175,-0.035478,0.495466
368,2021-10-25,4152.00,636457845,2.642573e+12,0.535365,0.535365,0.535365,0.510120,0.044192,0.505636
369,2021-10-26,4174.90,425736825,1.777409e+12,0.551541,0.551541,0.551541,0.451571,0.115892,0.513080
...,...,...,...,...,...,...,...,...,...,...
628,2022-11-11,2951.97,307688925,9.082885e+11,1.814890,1.814890,1.814890,0.552824,0.354259,0.548805
629,2022-11-14,2973.63,340500264,1.012522e+12,0.733747,0.733747,0.733747,0.577912,0.424156,0.560976
630,2022-11-15,3001.22,423046143,1.269655e+12,0.927822,0.927822,0.927822,0.596344,0.513823,0.576010
631,2022-11-16,3013.87,443440944,1.336473e+12,0.421495,0.421495,0.421495,0.595281,0.515016,0.576050


In [33]:

print("preprocessing")
stock = data_preprocessing(stock)

datetime_list = stock['datetime']

print("S1 score")
Volatility_list = calcul_volatility(stock)
Volume_list = calcul_volume(stock)
S1_score = volatility_volume_score(Volatility_list, Volume_list)

print("S2 score")
S2_score = momentum_score(S1_score, stock)

print("FG score")
FG_score = calcul_FGscore(S1_score, S2_score)

print("final_data")
final_data = pd.DataFrame({'date': datetime_list[365:],
                      'close': stock['close'][365:],
                      'volume': stock['volume'][365:],
                      'multiple_volume': stock['multiple_volume'][365:],
                      'per': stock['per'][365:],
                      'Volatility_list': Volatility_list,
                      'Volume_list': Volume_list,
                      'S1_score': S1_score,
                      'S2_score': S2_score,
                      'FG_score': FG_score})


preprocessing
S1 score


/home/user/anaconda3/envs/taehoon_py37/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in log
  del sys.path[0]
/home/user/anaconda3/envs/taehoon_py37/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/user/anaconda3/envs/taehoon_py37/lib/python3.7/site-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


S2 score
FG score
final_data


In [14]:
print(len(stock))
print(len(datetime_list))
print(len(Volatility_list))
print(len(Volume_list))
print(len(S1_score))
print(len(S2_score))
print(len(FG_score))
print(len(final_data))

633
633
268
633
268
268
268
268
